We're going to load the combined demographic and turnout data, and  predict turnout for the 2017 local elections - More Data

In [1]:
#First set everthing up
import pyodbc
import graphlab as gl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import local #server information and credentials
from pullODBC import pullODBC
from pullODBCall import pullODBCall
from matplotlib.backends.backend_pdf import PdfPages
from __future__ import division # convert integer division to floating point
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['font.size'] = 7
plt.tight_layout;
plt.rc('xtick', labelsize=7) 
plt.rc('ytick', labelsize=7) 

Read in combined demographic and participation data from the June 2017 snapshots

In [2]:
combined_df = pd.read_csv('combined_updated.csv')

C:\Users\scott\Anaconda2\envs\gl-env\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print len(combined_df)
print combined_df.head()
print combined_df.columns

6771263
   Unnamed: 0  Unnamed: 0.1  birth_age gender_code race_code ethnic_code  \
0           0             0         79           M         W          NL   
1           1             1         60           M         W          NL   
2           2             2         43           M         W          NL   
3           3             3         55           M         W          NL   
4           4             4         58           F         W          NL   

  party_cd status_cd  zip_code birth_state  ...  2007 2008 2009 2010 2011  \
0      DEM         A     27253          NC  ...   0.0  1.0  0.0  1.0  0.0   
1      REP         A     27215          NC  ...   0.0  1.0  0.0  1.0  0.0   
2      REP         A     27302          NC  ...   0.0  1.0  1.0  1.0  0.0   
3      REP         A     27215          NC  ...   0.0  1.0  0.0  1.0  0.0   
4      DEM         I     27258          NC  ...   NaN  NaN  NaN  NaN  NaN   

  2012 2013 2014  2015  2016  
0  1.0  0.0  1.0   0.0   1.0  
1  1.0  0.

Change all the NaN's to 0

In [4]:
combined_df.fillna(value=0.0, method=None, axis=None, inplace=True, limit=None, downcast=None)

In [5]:
pd.set_option('max_columns',100)

Get rid of junk columns

In [6]:
del combined_df['Unnamed: 0']
del combined_df['Unnamed: 0.1']

Change all particpation columns to integer

In [7]:
years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
for year in years:
    combined_df[year] = combined_df[year].apply(int)

In [8]:
print combined_df.head()

   birth_age gender_code race_code ethnic_code party_cd status_cd  zip_code  \
0         79           M         W          NL      DEM         A     27253   
1         60           M         W          NL      REP         A     27215   
2         43           M         W          NL      REP         A     27302   
3         55           M         W          NL      REP         A     27215   
4         58           F         W          NL      DEM         I     27258   

  birth_state drivers_lic county_desc  registr_dt      ncid precinct_abbrv  \
0          NC           N    ALAMANCE  01/19/1959   AA34771             07   
1          NC           Y    ALAMANCE  04/09/1984   AA53687            12W   
2          NC           Y    ALAMANCE  04/02/1992   AA80986            10S   
3          NC           Y    ALAMANCE  09/15/2003  AA131587            03S   
4          NC           Y    ALAMANCE  05/19/2006  AA146650             13   

  municipality_abbrv last_name first_name  2007  2008  2

Change registr_dt to datetime and birth_age to float

In [9]:
combined_df['birth_age'] = combined_df['birth_age'].apply(float)

In [10]:
combined_df['registr_dt']=pd.to_datetime(combined_df.registr_dt, format='%m/%d/%Y')
print combined_df.head()

   birth_age gender_code race_code ethnic_code party_cd status_cd  zip_code  \
0       79.0           M         W          NL      DEM         A     27253   
1       60.0           M         W          NL      REP         A     27215   
2       43.0           M         W          NL      REP         A     27302   
3       55.0           M         W          NL      REP         A     27215   
4       58.0           F         W          NL      DEM         I     27258   

  birth_state drivers_lic county_desc registr_dt      ncid precinct_abbrv  \
0          NC           N    ALAMANCE 1959-01-19   AA34771             07   
1          NC           Y    ALAMANCE 1984-04-09   AA53687            12W   
2          NC           Y    ALAMANCE 1992-04-02   AA80986            10S   
3          NC           Y    ALAMANCE 2003-09-15  AA131587            03S   
4          NC           Y    ALAMANCE 2006-05-19  AA146650             13   

  municipality_abbrv last_name first_name  2007  2008  2009  2

Create datetime objects for the four election days to be used in prediction

In [11]:
election_day_2016 = pd.to_datetime('11/8/2016', dayfirst=False, yearfirst=False)
election_day_2015 = pd.to_datetime('11/3/2015', dayfirst=False, yearfirst=False)
election_day_2014 = pd.to_datetime('11/4/2014', dayfirst=False, yearfirst=False) 

Create a set of functions to set new columns to 'Y' (voted) 'N' (registered but did not vote) and 'U' (not registered)

In [12]:
def calc_voted_2016(x):
    if x['2016'] == 1:
        return 'Y'
    elif x['registr_dt'] <= election_day_2016:
        return 'N'
    else:
        return 'U'

In [13]:
def calc_voted_2015(x):
    if x['2015'] == 1:
        return 'Y'
    elif x['registr_dt'] <= election_day_2015:
        return 'N'
    else:
        return 'U'

In [14]:
def calc_voted_2014(x):
    if x['2014'] == 1:
        return 'Y'
    elif x['registr_dt'] <= election_day_2014:
        return 'N'
    else:
        return 'U'

In [15]:
combined_df['congressional']= combined_df.apply(calc_voted_2014, axis=1)
print "created congressional column"
combined_df['local']= combined_df.apply(calc_voted_2015, axis=1)
print "created local column"
combined_df['presidential']= combined_df.apply(calc_voted_2016, axis=1)
print "created presiendial column"

created congressional column
created local column
created presiendial column


Print the first 10 rows and examine for correctness

In [16]:
pd.set_option('max_columns', 30)
print combined_df.head(n=10)

   birth_age gender_code race_code ethnic_code party_cd status_cd  zip_code  \
0       79.0           M         W          NL      DEM         A     27253   
1       60.0           M         W          NL      REP         A     27215   
2       43.0           M         W          NL      REP         A     27302   
3       55.0           M         W          NL      REP         A     27215   
4       58.0           F         W          NL      DEM         I     27258   
5       50.0           M         W          UN      REP         A     27302   
6       82.0           M         W          NL      REP         A     27302   
7       48.0           M         W          UN      UNA         A     27253   
8       62.0           F         W          NL      REP         A     27217   
9       30.0           F         W          NL      UNA         A     27253   

  birth_state drivers_lic county_desc registr_dt      ncid precinct_abbrv  \
0          NC           N    ALAMANCE 1959-01-19   AA

Before predicting, need to make sure the columns are of the correct type and that all folks with invalid ages and years registered are removed.  SFrames, which will be used for predicting, do not accept mixed types in columns, so we need to be very sure that all column types are the same.

In [17]:
string_columns = ['ncid', 'gender_code',  'race_code',  'ethnic_code', 'party_cd', 'status_cd', 
                'zip_code', 'birth_state', 'drivers_lic', 'county_desc', 'precinct_abbrv',
                 'congressional', 'local', 'presidential', 'municipality_abbrv', 'last_name' , 'first_name']

In [18]:
for col in string_columns: #convert all the string_columns to 'string'
    combined_df[col] = combined_df[col].apply(str)

Create interaction variables

In [19]:
def convert_to_years(x): # function to convert a timedelta object to a float of years
    return x.total_seconds()/(60*60*24*365.2422)

In [21]:
combined_df['race_gender'] = combined_df['race_code'] + combined_df ['gender_code']
combined_df['race_ethnicity'] = combined_df['race_code'] + combined_df['ethnic_code']
combined_df['gender_ethnicity'] = combined_df['gender_code'] + combined_df ['ethnic_code']
combined_df['age_squared'] = (combined_df['birth_age'])**2
combined_df['age_cubed'] = (combined_df['birth_age'])**3
now = pd.to_datetime('today') # get today's data
combined_df['years_since_registration'] = now - combined_df['registr_dt'] # get days since registration as a timedelta object
combined_df['years_since_registration'] = combined_df['years_since_registration'].apply(convert_to_years) # convert to years
combined_df['years_squared'] = (combined_df['years_since_registration'])**2
combined_df['years_cubed'] = (combined_df['years_since_registration'])**3
combined_df['county_prec'] = combined_df['county_desc'] + combined_df['precinct_abbrv']

In [22]:
print combined_df.head()

   birth_age gender_code race_code ethnic_code party_cd status_cd zip_code  \
0       79.0           M         W          NL      DEM         A    27253   
1       60.0           M         W          NL      REP         A    27215   
2       43.0           M         W          NL      REP         A    27302   
3       55.0           M         W          NL      REP         A    27215   
4       58.0           F         W          NL      DEM         I    27258   

  birth_state drivers_lic county_desc registr_dt      ncid precinct_abbrv  \
0          NC           N    ALAMANCE 1959-01-19   AA34771             07   
1          NC           Y    ALAMANCE 1984-04-09   AA53687            12W   
2          NC           Y    ALAMANCE 1992-04-02   AA80986            10S   
3          NC           Y    ALAMANCE 2003-09-15  AA131587            03S   
4          NC           Y    ALAMANCE 2006-05-19  AA146650             13   

  municipality_abbrv last_name     ...      2014  2015  2016  congre

By definition of status_cd = 'I', and municipality_abbrv = 'None' voters that have either of these attributes should be predicted as not voting in the next local election.  Therefore, the logical thing for our model to do is to predict all voters with status_cd = 'I' or municpality_abbrv = 'None' will not vote in the next local election. Our model only modeled the voters with status_cd = 'A'. We now combine everything in our final classification model to predict whether a voter will vote in the next municipal election:

    if status_cd == 'I' then predict 'No'
    
        else if municipality_abbrv == 'None' then predict 'No'

        else if model_prediction == 'N' then predict 'No'
    
        else predict 'Yes'

Now we convert combined_df to an SFrame for prediction.

In [23]:
combined_sf = gl.SFrame(combined_df)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\scott\AppData\Local\Temp\graphlab_server_1503508458.log.0


This non-commercial license of GraphLab Create for academic use is assigned to scottsmi@live.unc.edu and will expire on May 15, 2018.


Let's read in the models for predicting turnout and sentiment

In [24]:
turnout_model = gl.load_model('enhanced_turnout_model')
sentiment_model = gl.load_model('full_party_model_file.mdl')

Let's first predict sentiment

In [25]:
def return_actual_or_predicted_party(x): # This function returns the party if affiliated, if not returns the predicted party
    if x['party_cd'] == "DEM":
        return"DEM"
    elif x['party_cd'] == "REP":
        return "REP"
    elif x['predict_positive'] == True:
        return "REP"
    else:
        return "DEM"
    

def get_model_sentiment_predictions(model, sframe, threshold):
    sframe['percent_republican'] = model.predict(sframe, output_type='probability') # add a column with the model generated probabilities
    sframe['predict_positive'] = sframe['percent_republican'] >= threshold # add a column that contains 'true' if a positive is predicted, a false otherwise
    sframe['actual_or_predicted_party']= sframe.apply(return_actual_or_predicted_party)# add this column to contain the actual or predicted party preference
    return sframe            

In [26]:
prediction_sf= get_model_sentiment_predictions(sentiment_model, combined_sf, 0.56)

In [27]:
prediction_sf.print_rows(num_rows=10, num_columns=50, max_column_width=30, max_row_width=80, output_file=None)

+-----------+-------------+-----------+-------------+----------+-----------+
| birth_age | gender_code | race_code | ethnic_code | party_cd | status_cd |
+-----------+-------------+-----------+-------------+----------+-----------+
|    79.0   |      M      |     W     |      NL     |   DEM    |     A     |
|    60.0   |      M      |     W     |      NL     |   REP    |     A     |
|    43.0   |      M      |     W     |      NL     |   REP    |     A     |
|    55.0   |      M      |     W     |      NL     |   REP    |     A     |
|    58.0   |      F      |     W     |      NL     |   DEM    |     I     |
|    50.0   |      M      |     W     |      UN     |   REP    |     A     |
|    82.0   |      M      |     W     |      NL     |   REP    |     A     |
|    48.0   |      M      |     W     |      UN     |   UNA    |     A     |
|    62.0   |      F      |     W     |      NL     |   REP    |     A     |
|    30.0   |      F      |     W     |      NL     |   UNA    |     A     |

Let's now predict participation in 2017

In [28]:
def return_predicted_participation(x): # This function returns 'N' if status_cd = 'I', if not returns the predicted participation
    if x['status_cd'] == "I":
        return 'N'
    elif x['municipality_abbrv'] == "None":
        return 'N'
    elif x['predict_participate'] == False:
        return 'N'
    else:
        return 'Y'
    

def get_predicted_participation(model, sframe, threshold):
    sframe['percent_participation'] = model.predict(sframe, output_type='probability') # add a column with the model generated probabilities
    sframe['predict_participate'] = sframe['percent_participation'] >= threshold # add a column that contains 'true' if a positive is predicted, a false otherwise
    sframe['predicted_participation']= sframe.apply(return_predicted_participation)# add this column to contains the predicted participation
    return sframe            

In [29]:
prediction_sf = get_predicted_participation(turnout_model, prediction_sf, 0.49)

In [31]:
prediction_sf.print_rows(num_rows=10, num_columns=50, max_column_width=30, max_row_width=80, output_file=None)

+-----------+-------------+-----------+-------------+----------+-----------+
| birth_age | gender_code | race_code | ethnic_code | party_cd | status_cd |
+-----------+-------------+-----------+-------------+----------+-----------+
|    79.0   |      M      |     W     |      NL     |   DEM    |     A     |
|    60.0   |      M      |     W     |      NL     |   REP    |     A     |
|    43.0   |      M      |     W     |      NL     |   REP    |     A     |
|    55.0   |      M      |     W     |      NL     |   REP    |     A     |
|    58.0   |      F      |     W     |      NL     |   DEM    |     I     |
|    50.0   |      M      |     W     |      UN     |   REP    |     A     |
|    82.0   |      M      |     W     |      NL     |   REP    |     A     |
|    48.0   |      M      |     W     |      UN     |   UNA    |     A     |
|    62.0   |      F      |     W     |      NL     |   REP    |     A     |
|    30.0   |      F      |     W     |      NL     |   UNA    |     A     |

In [32]:
prediction_sf.export_csv('prediction_sf.csv')

Let's look at some data to see if predictions make sense

In [33]:
prediction_df = prediction_sf.to_dataframe()
majority = prediction_df.groupby(['actual_or_predicted_party'])['actual_or_predicted_party'].count()/prediction_df.groupby(['actual_or_predicted_party'])['actual_or_predicted_party'].count().sum()
print majority
majority = prediction_df.groupby(['predicted_participation'])['predicted_participation'].count()/prediction_df.groupby(['predicted_participation'])['predicted_participation'].count().sum()
print majority

actual_or_predicted_party
DEM    0.520806
REP    0.479194
Name: actual_or_predicted_party, dtype: float64
predicted_participation
N    0.878872
Y    0.121128
Name: predicted_participation, dtype: float64


The party prediction is consistent with previous work.  Let's work on turnout.  First, let's calculate turnout for the 2015 local election

In [34]:
majority = prediction_df.groupby(['2015'])['2015'].count()/prediction_df.groupby(['2015'])['2015'].count().sum()
print majority

2015
0    0.929566
1    0.070434
Name: 2015, dtype: float64


Prediction is higher than actual 2015 turnout, which may indicate that some additional tuning is necessary for the model (e.g. moving the threshold a little higher)

Let's look at only voters in muncipalities

In [35]:
temp_df = prediction_df[prediction_df['municipality_abbrv']!= 'None']

In [36]:
majority = temp_df.groupby(['predicted_participation'])['predicted_participation'].count()/temp_df.groupby(['predicted_participation'])['predicted_participation'].count().sum()
print majority

predicted_participation
N    0.800476
Y    0.199524
Name: predicted_participation, dtype: float64


In [37]:
majority = temp_df.groupby(['2015'])['2015'].count()/temp_df.groupby(['2015'])['2015'].count().sum()
print majority

2015
0    0.887298
1    0.112702
Name: 2015, dtype: float64


Predicted particpation is signficantly higher than 2015 participation.  Now let's remove inactive voters.

In [38]:
temp_df = temp_df[temp_df['status_cd']== 'A']

In [39]:
majority = temp_df.groupby(['predicted_participation'])['predicted_participation'].count()/temp_df.groupby(['predicted_participation'])['predicted_participation'].count().sum()
print majority

predicted_participation
N    0.7588
Y    0.2412
Name: predicted_participation, dtype: float64


In [40]:
majority = temp_df.groupby(['2015'])['2015'].count()/temp_df.groupby(['2015'])['2015'].count().sum()
print majority

2015
0    0.864726
1    0.135274
Name: 2015, dtype: float64


Predicted particpation is almost double acual 2015.  In the future, we may want to further tune the model by increasing the threshold for predicting voter participation.

Lastly, let's look at voters that particpated in the 2016 presidential election.

In [41]:
temp_df = prediction_df[prediction_df['2016'] == 1]

In [42]:
majority = temp_df.groupby(['actual_or_predicted_party'])['actual_or_predicted_party'].count()/temp_df.groupby(['actual_or_predicted_party'])['actual_or_predicted_party'].count().sum()
print majority

actual_or_predicted_party
DEM    0.507418
REP    0.492582
Name: actual_or_predicted_party, dtype: float64


<img src="files/nc_ag_20016.jpg">